In [1]:
# Install packages
!pip install -U accelerate
!pip install -U transformers
!pip install datasets
!pip install torch
!pip install sentencepiece
!pip install numpy
# !pip install google-cloud-storage
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
# Imports
import torch
import transformers
import numpy as np
import pandas as pd

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)

In [8]:
# Model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = 'adenhaus/mt5-large-tata'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

In [ ]:
def split_verbalisation(text):
  split_text = text.split("Verbalisation: ") # For non-eng blueprints
  # split_text = text.split("): ")  # For eng blueprints

  if len(split_text) > 1:
    return split_text[1]
  else:
    return text

In [4]:
# Predict function
def generate_verbalisation(model, tokenizer, example):
    # source = example['linearized_input']
    # target = example['target']
    input_ids = tokenizer(example)["input_ids"]
    # input_ids = torch.LongTensor(input_ids).view(1, -1).to(model.device)
    input_ids = torch.LongTensor(input_ids).view(1, -1).to(device)
    generated_ids = model.generate(input_ids, max_new_tokens=200, repetition_penalty=1.2)
    prediction = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # prediction = split_verbalisation(prediction)

    print(prediction)

    return prediction

In [ ]:
# Load test set
df = pd.read_csv("test_full.csv", sep='\t')

In [ ]:
df = pd.read_csv("pt_test.csv", sep='\t')

In [5]:
df = pd.read_csv("ru.csv", sep='\t')

In [ ]:
df = pd.read_csv("eng_blueprints_dev.csv", sep='\t')

In [ ]:
len(df)

763

In [ ]:
df.iloc[35]['linearized_input']

'Attitudes toward Wife Beating | Percent of women and men age15-49 who believe that a husband is justified in beating his wife for the following reasons: | (Women, Wife burns the food, 14) (Men, Wife burns the food, 8) (Women, Wife argues with him, 21) (Men, Wife argues with him, 13) (Women, Wife goes out without telling him, 25) (Men, Wife goes out without telling him, 13) (Women, Wife neglects children, 25) (Men, Wife neglects children, 14) (Women, Wife refuses to have sex with him, 19) (Men, Wife refuses to have sex with him, 11) (Women, Any of these, 35) (Men, Any of these, 25)'

In [ ]:
df.iloc[35]['target']

'More than one-third of women (35%) and one-quarter of men agree that a husband is justified in beating his wife for at least one of these reasons: if she burns the food, argues with him, goes out without telling him, neglects the children, or refuses to have sex with him.'

In [ ]:
generate_verbalisation(model, tokenizer, df.iloc[35]['linearized_input'])

Blueprint: 6%. What is the percentage of children under 5 with diarrhoea in Swazilands? | Verbalisation (English): The percentage of children under 5 with diarrhoea, according to 5% of children under 5 with diarrhoea.


'Blueprint: 6%. What is the percentage of children under 5 with diarrhoea in Swazilands? | Verbalisation (English): The percentage of children under 5 with diarrhoea, according to 5% of children under 5 with diarrhoea.'

In [9]:
df = df[~df['example_id'].str.contains('-en-')]

In [10]:
# Generate output dataset for evaluation
out_df = pd.DataFrame(columns=['preds', 'refs'])
out_df['refs'] = df['target']
out_df['preds'] = df['linearized_input'].apply(lambda x: generate_verbalisation(model, tokenizer, x))
out_df['linearized_input'] = df['linearized_input']
out_df.to_csv('mt5-large-ru-preds.csv', sep='\t', index=False)

Ahụkwara otu n'ụzọ atọ (34%) nke households in Unguja South تمتلك a njia iptafi ti afya yao.
Ahụkwara otu n'ụzọ atọ (34%) nke households possèdent acesso kuhusu dozi uzazi.
Ahụkwara otu n'ụzọ atọ (34%) nke households in Zanzibar sont osùgbọn awọn ọmọ tówà ninu ìpín kwenye makazio.
Zanzibar, 98% nke wanawake nchini Tanzania have completed primary education.
Atụmatụ 24 montre que a percentage of women in the semi-circamare zone are currently employed, com 25% na coast and 31% na coast.
A nível de escolaridade, a taxa de mortalidade est ligeiramente superior em Karak (2,6).
Among os diferentes estados, a percentage of women who are pregnant or having their mothers have a child born in the years preceding the survey were least significant (7%).
A n'y a pratiquement diferença entre l'utilisation de modern methods e l'accès à medidas أخرى.


KeyboardInterrupt: 

In [ ]:
for dataset in ['ar', 'en', 'fr', 'ha', 'ig', 'pt', 'sw', 'yo']:
  df = pd.read_csv(dataset + '_test.csv', sep='\t')

  out_df = pd.DataFrame(columns=['preds', 'refs'])
  out_df['refs'] = df['target']
  out_df['preds'] = df['linearized_input'].apply(lambda x: generate_verbalisation(model, tokenizer, x))
  out_df['linearized_input'] = df['linearized_input']
  out_df.to_csv('mt5-large-' + dataset + '-preds.csv', sep='\t', index=False)